# Two models comparison

## Libraries

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from ultralytics import YOLO

## Code

In [ ]:
# Example category dictionary (adjust if your classes are different)
CLASS_NAMES = {
    0: 'class_0',
    1: 'class_1',
    2: 'class_2',
    3: 'class_3',
    4: 'class_4',
    5: 'class_5',
    6: 'class_6',
    7: 'class_7',
    8: 'class_8',
    9: 'class_9',
}
#----------------------------------------FUNCTIONS----------------------------------------#
def generate_summary_text(class_ids):
    """Generates a summary text with the count of detected objects."""
    class_counts = Counter(class_ids)
    summary_list = [
        f"{count} {CLASS_NAMES.get(class_id, str(class_id))}"
        for class_id, count in class_counts.items()
    ]
    summary = ", ".join(summary_list)
    return summary if summary else "No objects detected"

def visualize_predictions(image_path, model):
    """
    Runs model prediction for 'image_path' and returns:
    - The image with detections drawn (to display with matplotlib).
    - A summary text with the detected classes.
    """
    results = model(image_path)

    # Temporarily save the image with predictions
    output_path = "temp_prediction.png"
    # results[0].save() saves the image with drawn boxes/borders
    results[0].save(filename=output_path)

    # Load the resulting image for displaying with matplotlib (RGB format)
    pred_img = cv2.imread(output_path)
    pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)

    # Extract detected classes
    if results[0].boxes is not None:
        class_ids = [int(box.cls) for box in results[0].boxes]
    else:
        class_ids = []

    # Generate summary text
    summary = generate_summary_text(class_ids)

    return pred_img, summary

def process_images_two_models(folder_path1, folder_path2, model1, model2):
    """
    Processes the same images in two different folders with their respective models.
    Assumes both folders contain files with the same names 
    (e.g., 'image1.jpg' in both folders).
    It will display side by side (2 columns) the output of model1 and model2 for each image.
    """
    # Get the list of images from the first folder
    # Assumes .jpg; adjust if you use .png or another format
    image_files1 = [f for f in os.listdir(folder_path1) if f.lower().endswith('.jpg')]

    # Filter to ensure the image also exists in the second folder
    image_files = []
    for f in image_files1:
        if os.path.exists(os.path.join(folder_path2, f)):
            image_files.append(f)

    # Create the figure: n rows (one per image), 2 columns
    fig, axes = plt.subplots(len(image_files), 2, figsize=(16, 6 * len(image_files)))

    # If there's only one image, 'axes' won't be a list of lists but a single object
    if len(image_files) == 1:
        axes = [axes]  # Normalize to handle in the same way in the loop

    for i, img_file in enumerate(image_files):
        # Paths to the same image in different folders
        img_path1 = os.path.join(folder_path1, img_file)
        img_path2 = os.path.join(folder_path2, img_file)

        # Inference with model 1
        pred_img1, summary1 = visualize_predictions(img_path1, model1)
        # Inference with model 2
        pred_img2, summary2 = visualize_predictions(img_path2, model2)

        # Show model 1 image and summary on the left column
        axes[i][0].imshow(pred_img1)
        axes[i][0].set_title(f"Model 1 - {img_file}")
        axes[i][0].axis("off")
        axes[i][0].text(
            0.5, -0.1, summary1,
            ha='center', va='top',
            transform=axes[i][0].transAxes,
            fontsize=12
        )

        # Show model 2 image and summary on the right column
        axes[i][1].imshow(pred_img2)
        axes[i][1].set_title(f"Model 2 - {img_file}")
        axes[i][1].axis("off")
        axes[i][1].text(
            0.5, -0.1, summary2,
            ha='center', va='top',
            transform=axes[i][1].transAxes,
            fontsize=12
        )

    plt.tight_layout()
    plt.show()



#----------------------------------------EXAMPLE----------------------------------------#
# Model paths
MODEL_PATH1 = "model_1.pt"
MODEL_PATH2 = "model_2.pt"

# Load both models
model1 = YOLO(MODEL_PATH1)
model2 = YOLO(MODEL_PATH2)

# Folders with the "same" images
# (e.g., one with original images, the other with preprocessed images)
folder1 = "folder_1_path"
folder2 = "folder_2_path"

# Process the images and display results in two columns
process_images_two_models(folder1, folder2, model1, model2)
